In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install bcolz
 
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import bcolz  # to process the data from Glove File 
import pickle # to dump and load pretrained glove vectors 
import copy   # to make deepcopy of python lists and dictionaries
import operator
import numpy as np
from pandas import DataFrame # to visualize the glove word embeddings in form of Dat

     |████████████████████████████████| 1.5MB 8.5MB/s 
  Created wheel for bcolz: filename=bcolz-1.2.1-cp36-cp36m-linux_x86_64.whl size=2656151 sha256=037442c8047e51ab53a813c9c83ed3127a735050f3588423dee8f93ce3473804
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [ ]:
vectors = bcolz.open(f'/content/drive/My Drive/glove.6B/6B.300.dat')[:]
words = pickle.load(open(f'/content/drive/My Drive/glove.6B/6B.300_words.pkl', 'rb'))
word2idx = pickle.load(open(f'/content/drive/My Drive/glove.6B/6B.300_idx.pkl', 'rb'))
 
glove = {w: vectors[word2idx[w]] for w in words}

In [ ]:
glove_dframe = DataFrame(vectors, columns=range(1,51), index=words)
glove_dframe[100:110]

ValueError: ignored

In [ ]:
sos_index = word2idx['sos']
eos_index = word2idx['eos']
sos_swap_word = words[0]
eos_swap_word = words[1]
 
words[0], words[sos_index] = words[sos_index], words[0]
words[1], words[eos_index] = words[eos_index], words[1]
word2idx[sos_swap_word], word2idx['sos'] = word2idx['sos'], word2idx[sos_swap_word]
word2idx[eos_swap_word], word2idx['eos'] = word2idx['eos'], word2idx[eos_swap_word]

In [ ]:
word2idx = { k : v for k , v in sorted(word2idx.items(), key=operator.itemgetter(1))}

In [ ]:
class InputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = { k : v for k , v in sorted(word2idx.items(), key=operator.itemgetter(1))}
        self.word2count = { word : 1 for word in words }
        self.index2word = { i : word for word, i in word2idx.items() }
        self.n_words = 400001
 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
 
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
SOS_token = 0
EOS_token = 1
 
 
class OutputLang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "sos", 1: "eos"}
        self.n_words = 2  # Count SOS and EOS
 
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
 
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?',;:])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?1234567890\-',;:]]+", r" ", s)
    return s

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/My Drive/nunkish seq2seq/%s-%s_new.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        pairs = [list((p)) for p in pairs]
        input_lang = InputLang(lang1)
        output_lang = OutputLang(lang2)

    return input_lang, output_lang, pairs

In [ ]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH 


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    print(pairs[0])
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


In [ ]:
input_lang, output_lang, pairs = prepareData('eng', 'nun', False)
print(random.choice(pairs))

Reading lines...
Read 166998 sentence pairs
['i am working .', 'nun goqui ikarikoha .']
Trimmed to 30482 sentence pairs
Counting words...
Counted words:
eng 404474
nun 59245
['this transformation in the relation of forces has its source in objective processes .', 'inhu yurhiruqin akugiq okmuhhu pukkup , makuniqui niruqgammerriq hunnahuisu paquhhuir renhaqquha .']


In [ ]:
matrix_len = input_lang.n_words
 
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0
 
for i, word in enumerate(input_lang.word2index):
    try: 
        weights_matrix[i] = glove[word]
        words_found += 1
    except KeyError:
        weights_matrix[i] = np.random.normal(scale=0.6, size=(300, ))

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super(EncoderRNN, self).__init__()
        self.hidden_size = embedding_dim
 
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(weights_matrix))
 
# Note here we are loading the embedding layer with pretrained weights i.e weight matrix created 
# from glove vectors --^
 
        self.gru = nn.GRU(hidden_size, hidden_size)
 
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
 
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
 
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
 
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
 
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
 
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
 
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
 
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights
 
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, save_every = 10000, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0  # Reset every print_every
     
 
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()
 
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
 
        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
         
 
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
        
        if iter % save_every == 0:

            print('saving checkpoint {0}'.format(iter))

            torch.save({
            'iter': iter,
            'model_state_dict': encoder.state_dict(),
            'optimizer_state_dict': encoder_optimizer.state_dict(),
            'loss': print_loss_avg,
            }, '/content/drive/My Drive/nunkish seq2seq/seq2seq update/encoder_15_cp{0}.dict'.format(iter))

            torch.save({
            'iter': iter,
            'model_state_dict': attn_decoder.state_dict(),
            'optimizer_state_dict': decoder_optimizer.state_dict(),
            'loss': print_loss_avg,
            }, '/content/drive/My Drive/nunkish seq2seq/seq2seq update/decoder_15_cp{0}.dict'.format(iter))

In [ ]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
            torch.save(encoder1.state_dict(), '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/encoder_{0}.dict'.format(iter))
            torch.save(attn_decoder1.state_dict(), '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/decoder_{0}.dict'.format(iter))

            torch.save({
            'n_iters': n_iters,
            'model_state_dict': encoder1.state_dict(),
            'optimizer_state_dict': encoder_optimizer.state_dict(),
            'loss': plot_losses,
            }, '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/encoder_cp{0}.dict'.format(iter))

            torch.save({
            'n_iters': n_iters,
            'model_state_dict': attn_decoder1.state_dict(),
            'optimizer_state_dict': decoder_optimizer.state_dict(),
            'loss': plot_losses,
            }, '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/decoder_cp{0}.dict'.format(iter))

            

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [ ]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 300
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder, attn_decoder, 200000, print_every=2000, save_every = 40000)

2m 26s (- 241m 30s) (2000 1%) 6.8599
4m 41s (- 229m 42s) (4000 2%) 6.9493
6m 57s (- 224m 49s) (6000 3%) 7.0561
9m 13s (- 221m 32s) (8000 4%) 7.0035
11m 30s (- 218m 42s) (10000 5%) 7.0419
13m 51s (- 217m 2s) (12000 6%) 7.0381
16m 10s (- 214m 54s) (14000 7%) 7.0668
18m 30s (- 212m 53s) (16000 8%) 6.9688
20m 50s (- 210m 45s) (18000 9%) 6.8731
23m 11s (- 208m 44s) (20000 10%) 6.8559
25m 31s (- 206m 29s) (22000 11%) 6.8238
27m 50s (- 204m 9s) (24000 12%) 6.7572
30m 11s (- 202m 0s) (26000 13%) 6.7265
32m 30s (- 199m 43s) (28000 14%) 6.7695
34m 52s (- 197m 35s) (30000 15%) 6.7362
37m 12s (- 195m 18s) (32000 16%) 6.6621
39m 32s (- 193m 2s) (34000 17%) 6.6007
41m 53s (- 190m 50s) (36000 18%) 6.5910
44m 14s (- 188m 34s) (38000 19%) 6.5808
46m 33s (- 186m 15s) (40000 20%) 6.4910
saving checkpoint 40000
48m 58s (- 184m 13s) (42000 21%) 6.5361
51m 18s (- 181m 54s) (44000 22%) 6.4541
53m 38s (- 179m 36s) (46000 23%) 6.4633
55m 58s (- 177m 15s) (48000 24%) 6.4411
58m 18s (- 174m 55s) (50000 25%) 6.38

In [ ]:
torch.save(encoder1.state_dict(), '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/encoder.dict')
torch.save(attn_decoder1.state_dict(), '/content/drive/My Drive/nunkish seq2seq/training_checkpoints/decoder.dict')

#load model
#encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
#decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

#encoder.load_state_dict(torch.load('encoder.dict'))
#decoder.load_state_dict(torch.load('decoder.dict'))

In [ ]:
learning_rate=0.01
hidden_size = 512
#load checkpoints
encoder = EncoderRNN(input_lang.n_words, hidden_size).to(device)
encoder_optimizer = optim.SGD(encoder1.parameters(), lr=learning_rate)

checkpoint = torch.load('/content/drive/My Drive/nunkish seq2seq/seq2seq update/encoder_15_cp200000.dict')
encoder.load_state_dict(checkpoint['model_state_dict'])
encoder_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
iter = checkpoint['iter']
print_loss_avg, = checkpoint['loss']

encoder.eval()
# - or -
encoder.train()

###########################################################################################################################

attn_decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)
decoder_optimizer = optim.SGD(attn_decoder1.parameters(), lr=learning_rate)

checkpoint = torch.load('/content/drive/My Drive/nunkish seq2seq/seq2seq update/decoder_15_cp200000.dict')
attn_decoder.load_state_dict(checkpoint['model_state_dict'])
decoder_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
iter = checkpoint['iter']
print_loss_avg, = checkpoint['loss']

attn_decoder.eval()
# - or -
attn_decoder.train()

AttnDecoderRNN(
  (embedding): Embedding(322582, 512)
  (attn): Linear(in_features=1024, out_features=200, bias=True)
  (attn_combine): Linear(in_features=1024, out_features=512, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (gru): GRU(512, 512)
  (out): Linear(in_features=512, out_features=322582, bias=True)
)

In [ ]:
hidden_size = 300
trainIters(encoder, attn_decoder, 200000, print_every=2000, save_every = 40000)

2m 35s (- 256m 24s) (2000 1%) 4.2827
4m 57s (- 242m 47s) (4000 2%) 4.1450
7m 20s (- 237m 13s) (6000 3%) 4.2096
9m 42s (- 232m 52s) (8000 4%) 4.1405
12m 4s (- 229m 17s) (10000 5%) 4.1678
14m 25s (- 225m 59s) (12000 6%) 4.1666
16m 46s (- 222m 57s) (14000 7%) 4.0897
19m 9s (- 220m 14s) (16000 8%) 4.1615
21m 30s (- 217m 28s) (18000 9%) 4.1331
23m 53s (- 215m 3s) (20000 10%) 4.1041
26m 16s (- 212m 32s) (22000 11%) 4.0640
28m 38s (- 210m 0s) (24000 12%) 4.0790
31m 2s (- 207m 44s) (26000 13%) 4.0671
33m 23s (- 205m 8s) (28000 14%) 3.9717
35m 45s (- 202m 40s) (30000 15%) 3.9707
38m 8s (- 200m 15s) (32000 16%) 3.9407
40m 32s (- 197m 54s) (34000 17%) 4.0396
42m 54s (- 195m 26s) (36000 18%) 4.0140
45m 15s (- 192m 57s) (38000 19%) 4.0187
47m 37s (- 190m 29s) (40000 20%) 4.0887
saving checkpoint 40000
50m 2s (- 188m 15s) (42000 21%) 4.0298
52m 24s (- 185m 48s) (44000 22%) 4.0057
54m 47s (- 183m 25s) (46000 23%) 3.9770
57m 10s (- 181m 1s) (48000 24%) 4.0262
59m 32s (- 178m 37s) (50000 25%) 3.9969
61

KeyboardInterrupt: ignored

In [ ]:
evaluateRandomly(encoder, attn_decoder)

> now they are not allowed to sail more than five miles off shore .
= immeqaha ruhuqiq ikanha uinha puiq huqqi pinmihirru unapuhirrummuhaguhiqqui .
< ugukruq immeha 100 uqquha pihhuk . . . . . . . . . .

> the other was the report that kerrey had been involved in war crimes .
= pukkenka rokki sahhur rakkunruqiq ihamuhhuk onku yoshi .
< inhu sahhur sahhur yoshi yoshi mek . . . <EOS>

> i protested and they punched me in the face .
= nun uhukra ohikmmah hokigirruisiq on paruhhiq rahha gihhunuk .
< nun , onnui nun nun nun . <EOS>

> what is a rajini speech without story ?
= rahhiruhui iqquhu kusini moyyu ?
< kusini kusini kusini ? ? <EOS>

> the judge has postponed hearing till then .
= ihunuq hukruqirupuru guqurra giyukunuisui huqqi guihhaqquk nihimuhi .
< inhu guqurra guqurra guqurra panma unapuhi . . . . . . . . .

> but we have been on the retreat .
= unuq mingunruy yossummuhhaqqep .
< unuq nunruq nup nunruq . . . . . . . . . . .

> many in the tamil film field are showing interest in

In [ ]:
output_words, attentions = evaluate(
    encoder1, attn_decoder1, "i love you .")
plt.matshow(attentions.numpy())

In [ ]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder, attn_decoder, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)

In [ ]:
evaluateAndShowAttention("i love you .")